In [1]:
%load_ext autoreload
import tensorflow as tf
import edward as ed
from edward.models import Normal, Categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
X_data, y_data = datasets.load_digits(return_X_y=True)
X_data = X_data/X_data.max()
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3)

In [3]:
DIN = X_train.shape[1]
DOUT = len(set(y_train))
X = tf.placeholder(tf.float32, [None, DIN])
W = Normal(mu=tf.zeros([DIN, DOUT]), sigma=tf.ones([DIN, DOUT]))
b = Normal(mu=tf.zeros([DOUT]), sigma=tf.ones([DOUT]))
y = Categorical(logits=tf.matmul(X, W) + b)

qW = Normal(mu=tf.Variable(tf.random_normal([DIN, DOUT])),
            sigma=tf.nn.softplus(tf.Variable(tf.random_normal([DIN, DOUT]))))
qb = Normal(mu=tf.Variable(tf.random_normal([DOUT])),
            sigma=tf.nn.softplus(tf.Variable(tf.random_normal([DOUT]))))

In [4]:
y.get_shape(), W.get_shape()

(TensorShape([Dimension(None)]), TensorShape([Dimension(64), Dimension(10)]))

In [5]:
inference = ed.KLqp({W: qW}, data={X: X_train, y: y_train}) 

In [8]:
inference.run(n_samples=5, n_iter=10000)

10000/10000 [100%] ██████████████████████████████ Elapsed: 97s | Loss: 721.560


In [55]:
preds = []
for i in range(5):
    wv = qW.eval()
    bv = qb.eval()

    y_pred = (np.matmul(X_test, wv) + bv)
    preds.append(y_pred)

In [56]:
preds = np.array(preds)

In [66]:
(preds.mean(axis=0).argmax(axis=1) == y_test).std()

0.26782157318208777

In [12]:
y_pred.shape

(540, 10)

In [37]:
%autoreload 2
import models
model = models.load('logistic_regression')

In [38]:
model.fit(X_train, y_train)

100/100 [100%] ██████████████████████████████ Elapsed: 3s | Loss: 791.133


In [54]:
(model.predict(X_test) == y_test).mean()

0.90555555555555556